In [ ]:
import pandas as pd
tab = pd.read_csv('../../data/data_base_vi_newton.csv',sep=',')

In [ ]:
tab.head(5)

,K,tau,r,rho,kappa,theta,sigma,V0,price,vol_imp_Newton
0,1.214497,0.771871,0.068510,-0.373242,1.321388,0.375778,0.400263,0.144875,0.108144,0.464404
1,1.086394,1.036183,0.044263,-0.859734,1.022232,0.240374,0.182461,0.203592,0.168776,0.456125
2,0.783620,1.057629,0.027852,-0.068008,1.073017,0.138973,0.107428,0.248494,0.307951,0.452266
3,1.042193,0.785673,0.060315,-0.726463,1.190304,0.106374,0.115668,0.301845,0.170844,0.479567
4,0.977431,0.775806,0.014470,-0.147284,1.587320,0.491061,0.175250,0.165743,0.205396,0.550043


In [ ]:
max(tab['vol_imp_Newton'])

0.7274978237169086

In [ ]:
import numpy as np

In [ ]:
np.shape(tab)

(213917, 10)

In [ ]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train_dataset = tab.sample(frac=0.5, random_state=0)
test_dataset = tab.drop(train_dataset.index).sample(frac = 0.2, random_state=0)

In [ ]:
np.shape(train_dataset), np.shape(test_dataset)

((106958, 10), (21392, 10))

In [ ]:
train_labels = train_dataset.pop("vol_imp_Newton")
test_labels = test_dataset.pop("vol_imp_Newton")

In [ ]:
train_labels.head(5)

115477    0.395993
91958     0.371575
134075    0.479654
131161    0.467564
188269    0.386860
Name: vol_imp_Newton, dtype: float64

In [ ]:
train_dataset.head(5)

,K,tau,r,rho,kappa,theta,sigma,V0,price
115477,0.934142,0.951744,0.017602,-0.069792,0.652707,0.047183,0.341584,0.202696,0.191109
91958,1.200779,0.644221,0.076011,-0.530779,0.901733,0.253517,0.417326,0.128618,0.067837
134075,0.788710,1.036410,0.060218,-0.666699,0.396105,0.266762,0.329716,0.207761,0.327212
131161,0.725699,0.378659,0.090101,-0.441241,0.311351,0.212103,0.385267,0.191680,0.311177
188269,0.732647,0.350226,0.066061,-0.552733,1.073753,0.247650,0.256093,0.108026,0.290297


## PAPIER 1

In [ ]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(400, activation='relu',input_shape=[len(train_dataset.keys())]),
        tf.keras.layers.Dense(400, activation='relu'),
        tf.keras.layers.Dense(400, activation='relu'),
        tf.keras.layers.Dense(400, activation='relu'),
        tf.keras.layers.Dense(400, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(0.0001)
    
    model.compile(loss='mse', optimizer= optimizer ,metrics=['mae','mse'])
    
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               4000      
_________________________________________________________________
dense_1 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_3 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_4 (Dense)              (None, 400)               160400    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 401       
Total params: 646,001
Trainable params: 646,001
Non-trainable params: 0
__________________________________________________

In [ ]:
EPOCHS = 3000

history = model.fit(train_dataset, train_labels, epochs = EPOCHS, validation_data=(test_dataset, test_labels), batch_size=1024)

Epoch 1/1000
105/105 [==============================] - 1s 7ms/step - loss: 0.0125 - mae: 0.0642 - mse: 0.0125 - val_loss: 8.0829e-04 - val_mae: 0.0215 - val_mse: 8.0829e-04
Epoch 2/1000
105/105 [==============================] - 1s 5ms/step - loss: 7.8127e-04 - mae: 0.0211 - mse: 7.8127e-04 - val_loss: 7.6913e-04 - val_mae: 0.0210 - val_mse: 7.6913e-04
Epoch 3/1000
105/105 [==============================] - 1s 5ms/step - loss: 7.3681e-04 - mae: 0.0205 - mse: 7.3681e-04 - val_loss: 7.1128e-04 - val_mae: 0.0201 - val_mse: 7.1128e-04
Epoch 4/1000
105/105 [==============================] - 1s 5ms/step - loss: 6.8536e-04 - mae: 0.0197 - mse: 6.8536e-04 - val_loss: 6.6024e-04 - val_mae: 0.0194 - val_mse: 6.6024e-04
Epoch 5/1000
105/105 [==============================] - 1s 5ms/step - loss: 6.2981e-04 - mae: 0.0189 - mse: 6.2981e-04 - val_loss: 6.0143e-04 - val_mae: 0.0185 - val_mse: 6.0143e-04
Epoch 6/1000
105/105 [==============================] - 1s 5ms/step - loss: 5.6669e-04 - mae: 0.01

## Heston

In [ ]:
from scipy import integrate
import numpy as np

def phi(u,kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_):
    gamma_ = np.sqrt((sigma_**2)*(u**2+u*1j)+(kappa_-1j*rho_*sigma_*u)**2)
    a = 1j*u*np.log(S0_) + 1j*u*r_*tau_+kappa_*theta_*tau_*(kappa_-1j*rho_*sigma_*u)/(sigma_**2)
    b = (u**2+1j*u)*V0_/(gamma_*np.cosh(gamma_*tau_/2)/np.sinh(gamma_*tau_/2)+kappa_-1j*rho_*sigma_*u)
    c = np.cosh(gamma_*tau_/2)+((kappa_-1j*rho_*sigma_*u)/gamma_)*np.sinh(gamma_*tau_/2)
    d = 2*kappa_*theta_/(sigma_**2)
    return np.exp(a)*np.exp(-b)/(c**d)

def psi(nu_,alpha_,K_,r_,tau_,kappa_,S0_,theta_,rho_,sigma_,V0_):
    k_ = np.log(K_)
    F = phi(nu_-1j*(alpha_+1),kappa_,S0_,r_,tau_,theta_,rho_,sigma_,V0_)*np.exp(-1j*nu_*k_)
    d = (alpha_+1j*nu_)*(alpha_+1+1j*nu_)
    return np.exp(-r_*tau_-alpha_*k_)/np.pi*(F/d).real

import scipy as scp

def C(K_,alpha_,r_,tau_,kappa_,S0_,theta_,rho_,sigma_,V0_,L_):
    I = scp.integrate.quad(lambda nu_: psi(nu_,alpha_,K_,r_,tau_,kappa_,S0_,theta_,rho_,sigma_,V0_) , 0, L_)
    return I[0]

In [ ]:
r = [0.02]*(7*6)
rho = [-0.5]*(7*6)
kappa = [1.5]*(7*6)
sigma = [0.3]*(7*6)
theta = [0.1]*(7*6)
V0 = [0.1]*(7*6)

In [ ]:
tmp_K = np.linspace(0.7,1.3,7)
tmp_tau = np.linspace(0.5,1,6)

P = []
K = []
tau = []
for k in range(len(tmp_K)):
    for t in range(len(tmp_tau)):
      P.append(C(tmp_K[k],1,0.02,tmp_tau[t],1.5,1,0.1,-0.5,0.3,0.1,200))
      K.append(tmp_K[k])
      tau.append(tmp_tau[t])

In [ ]:
df = pd.DataFrame({'K': K, 'tau' : tau, 'r': r, 'rho': rho, 'kappa': kappa,'theta' : theta,'sigma': sigma, 'V0' : V0, 'price' : P})
df.head(5)

,K,tau,r,rho,kappa,theta,sigma,V0,price
0,0.7,0.5,0.02,-0.5,1.5,0.1,0.3,0.1,0.312992
1,0.7,0.6,0.02,-0.5,1.5,0.1,0.3,0.1,0.316809
2,0.7,0.7,0.02,-0.5,1.5,0.1,0.3,0.1,0.320703
3,0.7,0.8,0.02,-0.5,1.5,0.1,0.3,0.1,0.324618
4,0.7,0.9,0.02,-0.5,1.5,0.1,0.3,0.1,0.328520


In [ ]:
test_predictions = model.predict(df).flatten()

In [ ]:
test_predictions

array([0.34533116, 0.34338266, 0.3416982 , 0.3400709 , 0.33845714,
       0.3369742 , 0.33262357, 0.33111146, 0.32986528, 0.32869548,
       0.32749254, 0.32646632, 0.32067534, 0.3196754 , 0.31901354,
       0.31837708, 0.3176531 , 0.31703013, 0.31060562, 0.3100294 ,
       0.30975938, 0.30949506, 0.30910745, 0.3087436 , 0.30206126,
       0.301908  , 0.3019751 , 0.3020549 , 0.3019637 , 0.30183572,
       0.29469764, 0.2951265 , 0.29559928, 0.29603708, 0.2961905 ,
       0.2962038 , 0.2882428 , 0.28904307, 0.28982142, 0.29050398,
       0.2909686 , 0.29131562], dtype=float32)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

669/669 - 1s - loss: 1.2456e-06 - mae: 9.3257e-04 - mse: 1.2456e-06
Testing set Mean Abs Error:  0.00 MPG


In [ ]:
submission = pd.DataFrame({'K': K, 'tau' : tau, 'r': r, 'rho': rho, 'kappa': kappa, 'theta' : theta,'sigma': sigma, 'V0' : V0, 'price' : P,'vol_imp' : test_predictions})
submission.head()
submission.to_csv("prediction_1000epochs_elu.csv", index=False)

In [ ]:
from google.colab import files

files.download("prediction_1000epochs_elu.csv")